In [1]:
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
from nltk.chunk.regexp import RegexpParser
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from bs4 import BeautifulSoup
from collections import Counter, Iterable
from bs4.element import Comment
import requests, re, urllib, warnings, operator
from urllib.parse import urlparse
from urllib.error import HTTPError
from nltk.corpus import wordnet as wn

In [2]:

def checkAppropriateURL(url):
    parsedURL = urlparse(url)
    extn = parsedURL.path.split(".")[-1]
    site = parsedURL.netloc.split(".")
    path = parsedURL.path.split("/")[-1]
    if extn in ["pdf","doc","docx","xls","xlsx","ppt","pptx","odt"]:
        return False
    if any(i in site for i in ["pinterest"]):
        return False
    try:
        r = dict(requests.head(url).headers)["Content-Type"].split(";")[0]
        if(not r=="text/html"):
            print ("Failed",url,r)
            return False
    except:
        pass
    return True

def scrapeGoogleForAbstracts(query, count):
    words = query.split()
    q = ("+").join(words)
    print (q)
    gurl = "https://www.google.co.in/search?q=\"" + q + "\""
    data = requests.get(gurl).text
    soup = BeautifulSoup(data, "lxml")
    h3Rows = soup.find_all("h3", {"class":"r"})
    spanRows = soup.find_all("span", {"class":"st"})
    zippedRows = zip(h3Rows,spanRows)

    urls = []
    for zippedRow in zippedRows:
        try:
            h3Row = zippedRow[0]
            url = h3Row.find("a")['href']
            title = h3Row.text
            if(url.split("?")[0]=="/search"):
                continue
        except (GeneratorExit, KeyboardInterrupt, SystemExit):
            raise
        except:
            continue
        par = urllib.parse.parse_qs(urlparse(url).query)
        print ("par",par)
        try:
            appendingURL = par['q'][0]
        except:
            continue
        if(not checkAppropriateURL(appendingURL)):
            continue
        matchedQuery = query
        allbElems = zippedRow[1].findAll('b')
        for bElem in allbElems:
            if bElem.string[0].isalnum():
                matchedQuery = bElem.string
        urls.append((matchedQuery,appendingURL,title))
    i = 1
    same = 0
    prev = 0
    while(len(urls)<count):
        # print (len(urls),number)
        tempurl = gurl+ "&start=" +str(i*10)
        i+=1
        data = requests.get(tempurl).text
        soup = BeautifulSoup(data, "lxml")
        h3Rows = soup.find_all("h3", {"class":"r"})
        spanRows = soup.find_all("span", {"class":"st"})
        zippedRows = zip(h3Rows,spanRows)
        for (j,zippedRow) in enumerate(zippedRows):
            try:
                h3Row = zippedRow[0]
                url = h3Row.find("a")['href']
                title = h3Row.text
            except (GeneratorExit, KeyboardInterrupt, SystemExit):
                raise
            except:
                continue
            par = urllib.parse.parse_qs(urlparse(url).query)
            try:
                appendingURL = par['q'][0]
                if(not checkAppropriateURL(appendingURL)):
                    continue
                matchedQuery = query
                allbElems = zippedRow[1].findAll('b')
                for bElem in allbElems:
                    if bElem.string[0].isalnum():
                        matchedQuery = bElem.string
                urls.append((matchedQuery, appendingURL, title))
            except (GeneratorExit, KeyboardInterrupt, SystemExit):
                raise
            except:
                continue
        if prev == len(urls):
            same +=1 
            if same>=10:
                same = 0
                break
        prev = len(urls)
    return urls[:count]

def getCluesPatternsTuple(word):
    patterns = []
    patterns.append(("such as " + word, False))
    patterns.append((word + " and other", True))
    patterns.append((word + " or other", True))
    patterns.append(("including " + word, False))
    patterns.append(("especially " + word, False))
    return (patterns)


def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body):
    # Scraping a Web Document

    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

def getTextFromURL(url):
    hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
   'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
   'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
   'Accept-Encoding': 'none',
   'Accept-Language': 'en-US,en;q=0.8',
   'Connection': 'keep-alive'}
    # print (url)
    html = urllib.request.urlopen(urllib.request.Request(url, headers=hdr))
    inputText = text_from_html(html)
    return inputText

def getAbstracts(query, count):
    urls = scrapeGoogleForAbstracts(query, 10)
    # print ("lenght",len(urls))
    texts = []
    for (q,url,title) in urls:
        print (url)
        try:
            text = getTextFromURL(url)
        except (GeneratorExit, KeyboardInterrupt, SystemExit):
            raise
        except:
            continue
        texts.append((q,url,text.strip(),title))
    return texts


In [3]:

def flatten(l):
    for el in l:
        if isinstance(el, Iterable) and not isinstance(el, (str, bytes)):
            yield from flatten(el)
        else:
            yield el

def getHypernymsFromWordnet(word):
    allSynsets = wn.synsets(word)
    multiList = [[[a.name() for a in b.lemmas()] for b in c.hypernyms()]for c in wn.synsets(word)]
    finalList = list(flatten(multiList))
    return finalList

def getInstances(text):
    grammar = """
        PRE:   {<NNS|NNP|NN|NP|JJ|UH>+}
        MID: {<DT|IN|POS|FW|-|NP|NPS|NN|NNS>+}
        INSTANCE:   {(<DT+>)?(<JJ+>)?<PRE>(<MID><PRE>)?}
    """
    chunker = RegexpParser(grammar)
    taggedText = pos_tag(word_tokenize(text))
    textChunks = chunker.parse(taggedText)
    current_chunk = []
    for i in textChunks:
        if (type(i) == Tree and i.label() == "INSTANCE"):
            # print (i.leaves())
            current_chunk.append(" ".join([token for token, pos in i.leaves()]))
    return current_chunk

def getConcepts(text):
    grammar = """
        CONCEPT:   {(<DT>)?(<JJ>)?<NN|NNS>+}
    """
    chunker = RegexpParser(grammar)
    taggedText = pos_tag(word_tokenize(text))
    textChunks = chunker.parse(taggedText)
    current_chunk = []
    for i in textChunks:
        if (type(i) == Tree and i.label() == "CONCEPT"):
            current_chunk.append(" ".join([token for token, pos in i.leaves()]))
    return current_chunk

stopwords_en = stopwords.words("english")

def preprocessing(raw):
    wordlist = word_tokenize(raw)
    text = [w.lower () for w in wordlist if w not in stopwords_en]
    return text

def simDoc(text1, text2):
    word_set = set(text1).union(set(text2))

    freqd_text1 = FreqDist(text1)
    text1_count_dict = dict.fromkeys(word_set, 0)
    for word in text1:
        text1_count_dict[word] = freqd_text1[word]

    freqd_text2 = FreqDist(text2)
    text2_count_dict = dict.fromkeys(word_set, 0)
    for word in text2:
        text2_count_dict[word] = freqd_text2[word]

    taggeddocs = []
    doc1 = TaggedDocument(words = text1, tags = [u'NEWS_1'])
    taggeddocs.append(doc1)
    doc2 = TaggedDocument(words = text2, tags = [u'NEWS_2'])
    taggeddocs.append(doc2)

    #build the modwl
    model = Doc2Vec(taggeddocs, dm =0, alpha=0.025, vector_size=20, min_alpha=0.025, min_count=0)

    model.train(taggeddocs, epochs=model.iter, total_examples=model.corpus_count)
    print ("FF",len(text1),len(text2))
    similarity = model.n_similarity(text1, text2)
    return similarity

In [4]:
def extractConcepts(posList, text, query):
    allInstances = []
    for (st,end) in posList:
        if query[1]:
            #Instances after, and exists
            tempText = text[end+1:]
            rest = re.match(".*?\.",tempText)
            if rest:
                rest = rest.group()
            else:
                rest = tempText
            andExists = re.search(" (and|or) ", rest)
            if andExists:
                print ("rest:", rest)
                lastConceptPos = andExists.end()
                lastConcept = rest[lastConceptPos:]
                print ("lastConcept: ", lastConcept)
                lastConceptExtracted = getConcepts(lastConcept.strip())
                if lastConceptExtracted:
                    allInstances.append(lastConceptExtracted[0])
                prevConceptsEnd = andExists.start()
                prevConcepts = rest[:prevConceptsEnd]
                prevConceptsList = prevConcepts.split(",")
                prevConceptsList = [a for a in prevConceptsList if a.strip()]
                print ("****START")
                print ("previous Concepts:",prevConcepts)
                for concept in prevConceptsList:
                        print (concept.strip(), getConcepts(concept.strip()))
                        conceptExtracted = getConcepts(concept.strip())
                        if conceptExtracted:
                            allInstances.append(conceptExtracted[-1])
                print ("*****DONE")
            else:
                allConcepts = rest.split(",")
                print ("#####HERE ARE ALL CONCEPTS:",allConcepts)
                for (i,concept) in enumerate(allConcepts):
                        if i==(len(allConcepts)-1):
                            print (concept)
                            conceptExtracted = getConcepts(concept.strip())
                            print ("1######CONCEPT EXTRACTED:", conceptExtracted)
                            if conceptExtracted:
                                allInstances.append(conceptExtracted[0])
                        else:
                            conceptExtracted = getConcepts(concept.strip())
                            print ("######CONCEPT EXTRACTED:", conceptExtracted)
                            if conceptExtracted:
                                allInstances.append(conceptExtracted[-1])
        else:
            tempText = text[:st]
            print ("tempText", tempText)
            try:
                lastSentPos = [m.end() for m in re.finditer(".*?\.",tempText)][-1]
                lastSent = tempText[:lastSentPos]
            except:
                lastSent = tempText
            lastSentList = lastSent.split(",")
            lastSentList = [a for a in lastSentList if a.strip()]
            if len(lastSentList) == 1:
                conceptExtracted = getConcepts(lastSentList[0].strip())
                if conceptExtracted:
                    allInstances.append(conceptExtracted[-1])
            else:
                for (i,concept) in enumerate(lastSentList):
                    conceptExtracted = getConcepts(concept.strip())
                    if conceptExtracted:
                        if i==(len(lastSentList)-1):
                            allInstances.append(conceptExtracted[0])
                        else:
                            allInstances.append(conceptExtracted[-1])
    return allInstances

In [5]:
def getHypernymsFromWWW(instance):
    cluesPatternsTuple = getCluesPatternsTuple(instance)
    hits = {}
    for c,(clue,isAfter) in enumerate(cluesPatternsTuple):
         
        allAbstracts = getAbstracts(clue,10)
        # print (allAbstracts)
        for (query,url,abstract,title) in allAbstracts:
            filename = ''.join(e for e in url if e.isalnum())
            f = open(filename[:15]+".txt","w+")
            f.write(abstract)
            f.close()

        for cnt,(query,url,abstract,title) in enumerate(allAbstracts):
            print (len(abstract))
            text1 = preprocessing(abstract)
            text2 = preprocessing(text)
            print ("ssF",len(text1),len(text2))
            if(not len(text1)):
                continue
            similarity = simDoc(text1, text2)

            if similarity>threshold:
                print (c,"::",cnt,":",url)
                startEndPositions = []
                try:
                    for m in re.finditer(re.escape(query), abstract):
                        try:
                            startEndPositions.append((m.start(), m.end()))
                        except:
                            print ("query:",query,"url:",url)
                            titleSearch = re.search(clue, title)
                            if titleSearch:
                                startEndPositions.append((titleSearch.start(), titleSearch.end()))
                            else:
                                continue
                except:
                    print ("1","query:",q,"url:",url)
                    continue
                allConcepts = extractConcepts(startEndPositions, abstract, query)
                if(allConcepts):
                    for conc in allConcepts:
                        if conc in hits:
                            hits[conc] += 1
                        else:
                            hits[conc] = 1
                print ("Number,",c," All concepts:", allConcepts)
    
    print ("Done for instance", instance)
    concept = max(hits.items(), key=operator.itemgetter(1))[0]
    return (concept, hits)


In [9]:

def URLInput():
    url = 'https://www.webstaurantstore.com/article/101/types-of-pizza.html'
    text = getTextFromURL(url)
    return text

def DocInput():
    file = open("test.txt")
    text = file.read()
    return text



def main(text, threshold):
#     instances = getInstances(text)
    instances = ["vulnerability", "threat"]
    finalList = []
    for instance in instances:
        #Add Code for iterating through patterns and clues here
        
        concept = getHypernymsFromWWW(instance)
        finalList.append((concept,instance))    

        concept = getHypernymsFromWordnet(instance)
        finalList.append((concept,instance))
        print ("Doing instance",instance)
    print ("Here's the final list of instances: ",finalList)
    return finalList

# text = URLInput()
text = DocInput()
threshold = 0.3
hypernymsDict = main(text, threshold)

such+as+vulnerability
par {'q': ['https://www.kaspersky.co.in/small-to-medium-business-security/systems-management'], 'sa': ['U'], 'ved': ['0ahUKEwjNi_78-ZrhAhXRbX0KHZ_QD6YQFggUMAA'], 'usg': ['AOvVaw1Yx1wnOCC9cXiCBzl9Wmzr']}
par {'q': ['https://www.sciencedirect.com/topics/computer-science/vulnerability-database'], 'sa': ['U'], 'ved': ['0ahUKEwjNi_78-ZrhAhXRbX0KHZ_QD6YQFggaMAE'], 'usg': ['AOvVaw0ZQo1NPLIUK_2FPQA6oQNc']}
par {'q': ['https://www.sciencedirect.com/topics/computer-science/vulnerability-pattern'], 'sa': ['U'], 'ved': ['0ahUKEwjNi_78-ZrhAhXRbX0KHZ_QD6YQFggcMAI'], 'usg': ['AOvVaw3BZjTZ8WYIm3hHcsZHjrR-']}
par {'q': ['https://unit42.paloaltonetworks.com/unit-42-vulnerability-research-team-discovers-23-new-vulnerabilities-february-2019-disclosures-adobe-and-microsoft/'], 'sa': ['U'], 'ved': ['0ahUKEwjNi_78-ZrhAhXRbX0KHZ_QD6YQFggeMAM'], 'usg': ['AOvVaw3wet4h43oc7TYJpyXVrYAN']}
par {'q': ['https://www.banglajol.info/index.php/JLES/article/view/20150/13929'], 'sa': ['U'], 'ved': ['

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://www.imperva.com/blog/the-state-of-web-application-vulnerabilities-in-2018/
https://www.manageengine.com/vulnerability-management/vulnerability-assessment.html
https://www.bsimm.com/framework/deployment/configuration-and-vulnerability-management.html
https://www.netsparker.com/online-vulnerability-scanner/website-vulnerability-scanner-online/
https://www.csoonline.com/article/3323412/risk-based-vulnerability-management-a-better-form-of-cyber-defense.html
0
ssF 0 11967
12516
ssF 1481 11967


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:73: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:75: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
/usr/local/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


FF 1481 11967
0 :: 1 : https://www.sciencedirect.com/topics/computer-science/vulnerability-database
rest: databases , security communities, and so on.
lastConcept:  so on.
****START
previous Concepts: databases , security communities,
databases ['databases']
security communities ['security communities']
*****DONE
Number, 0  All concepts: ['databases', 'security communities']
35855
ssF 4215 11967
FF 4215 11967
0 :: 2 : https://www.sciencedirect.com/topics/computer-science/vulnerability-pattern
#####HERE ARE ALL CONCEPTS: ['patterns .']
patterns .
1######CONCEPT EXTRACTED: ['patterns']
Number, 0  All concepts: ['patterns']
4955
ssF 551 11967
FF 551 11967
17603
ssF 3362 11967
FF 3362 11967
0 :: 4 : https://www.banglajol.info/index.php/JLES/article/view/20150/13929
Number, 0  All concepts: []
13377
ssF 1637 11967
FF 1637 11967
0 :: 5 : https://www.imperva.com/blog/the-state-of-web-application-vulnerabilities-in-2018/
rest: databases, newsletters, forums, social media and more, integrates i

...

ssF 1009 11967
FF 1009 11967
3 :: 2 : https://chsiconnections.com/including-vulnerability-and-velocity-in-risk-assessments/
rest: can better allow companies or public risk pools to gauge the extent to which risk exposure exists for a given event and make proportionate provisions to protect the entities’ assets accordingly.
lastConcept:  public risk pools to gauge the extent to which risk exposure exists for a given event and make proportionate provisions to protect the entities’ assets accordingly.
****START
previous Concepts: can better allow companies
can better allow companies ['companies']
*****DONE
rest: and velocity in a risk management program not only better aligns an insurer’s position as far as exposure to the potential for disaster, it helps in the strategic planning process for response and mitigation and should be included when considering insurance software solutions for the future.
lastConcept:  mitigation and should be included when considering insurance software soluti

Doing instance vulnerability
such+as+threat
par {'q': ['https://en.wikipedia.org/wiki/Threat_(computer)'], 'sa': ['U'], 'ved': ['0ahUKEwjp59Ly-prhAhXYXisKHW5oCzcQFggUMAA'], 'usg': ['AOvVaw11Uz5xXP3NbFAB8_h5xXAM']}
par {'q': ['https://resources.infosecinstitute.com/category/enterprise/threat-hunting/threat-hunting-benefits/threat-hunting-vs-siem/'], 'sa': ['U'], 'ved': ['0ahUKEwjp59Ly-prhAhXYXisKHW5oCzcQFggaMAE'], 'usg': ['AOvVaw2wVXESSbtwNPLiIDycGXXe']}
par {'q': ['https://www.sans.org/course/cyber-threat-intelligence'], 'sa': ['U'], 'ved': ['0ahUKEwjp59Ly-prhAhXYXisKHW5oCzcQFggfMAI'], 'usg': ['AOvVaw0sahpD-4Xd_rPeSQ2IIgMK']}
par {'q': ['https://www.mdpi.com/1999-5903/11/3/63/htm'], 'sa': ['U'], 'ved': ['0ahUKEwjp59Ly-prhAhXYXisKHW5oCzcQFgglMAM'], 'usg': ['AOvVaw34jFGWhI_Ln9S8tu4trW3F']}
par {'q': ['https://www.pandasecurity.com/mediacenter/adaptive-defense/threat-hunting-reasons-necessary/'], 'sa': ['U'], 'ved': ['0ahUKEwjp59Ly-prhAhXYXisKHW5oCzcQFggrMAQ'], 'usg': ['AOvVaw02shdyCfR0D3

FF 671 11967
7539
ssF 910 11967
FF 910 11967
1 :: 5 : https://www.forbes.com/sites/quickerbettertech/2017/04/16/an-amazon-hacking-threat-and-other-small-business-tech-news-this-week/
Number, 1  All concepts: []
5152
ssF 741 11967
FF 741 11967
1 :: 6 : https://www.youtube.com/watch?v=VYoSxJWCwtE
#####HERE ARE ALL CONCEPTS: ['allegations                 ABP ANANDA               Loading.']
allegations                 ABP ANANDA               Loading.
1######CONCEPT EXTRACTED: ['allegations']
#####HERE ARE ALL CONCEPTS: ['allegations     Category   News & Politics       Show more  Show less       Loading.']
allegations     Category   News & Politics       Show more  Show less       Loading.
1######CONCEPT EXTRACTED: ['allegations']
Number, 1  All concepts: ['allegations', 'allegations']
6622
ssF 894 11967
FF 894 11967
1 :: 7 : https://www.youtube.com/watch?v=ms4h-nXKrhg
Number, 1  All concepts: []
6907
ssF 787 11967
FF 787 11967
8893
ssF 943 11967
FF 943 11967
threat+or+other
par {'q': ['h

par {'q': ['https://www.eccouncil.org/wp-content/uploads/2016/07/CTIA-Brochure.pdf'], 'sa': ['U'], 'ved': ['0ahUKEwipwu-3-5rhAhXQQ30KHS4cAUIQFgg3MAk'], 'usg': ['AOvVaw3H8JU17MxiyK49IN-xzcPd']}
https://ieeexplore.ieee.org/document/7684094
https://www.securonix.com/securonix-threat-research-lab/
https://www.semanticscholar.org/paper/Including-threat-actor-capability-and-motivation-in-Rossebo-Fransen/3e6767a3383e784f5198c5ee575a85caeeac3af8/figure/0
https://docs.microsoft.com/en-us/windows-hardware/drivers/driversecurity/threat-modeling-for-drivers
https://www.researchgate.net/publication/309498974_Including_threat_actor_capability_and_motivation_in_risk_assessment_for_Smart_GRIDs
https://www.healthleadersmedia.com/clinical-care/ohio-doctor-fired-after-anti-semitic-tweets-surface-including-threat-give-jews-wrong
https://www.nbcnews.com/news/us-news/ohio-doctor-fired-after-anti-semitic-tweets-surface-including-threat-n953916
https://www.cloudera.com/solutions/gallery/fireeye-threat-intelli

ssF 3080 11967
FF 3080 11967
3459
ssF 412 11967
FF 412 11967
3 :: 4 : https://www.healthleadersmedia.com/clinical-care/ohio-doctor-fired-after-anti-semitic-tweets-surface-including-threat-give-jews-wrong
#####HERE ARE ALL CONCEPTS: ["to give Jews 'wrong meds'  By NBC News  | January 03", ' 2019            Topics        patient safety      physicians      accountability      physicians management      prescription drugs                            An Ohio doctor accused of making anti-Semitic remarks on social media', ' including a tweet in which she allegedly said she would give Jewish people the wrong medication', ' was fired from the medical center she worked at', ' officials said.']
######CONCEPT EXTRACTED: ['meds']
######CONCEPT EXTRACTED: ['patient safety physicians', 'physicians management prescription drugs', 'doctor', 'anti-Semitic remarks', 'social media']
######CONCEPT EXTRACTED: ['a tweet', 'Jewish people', 'the wrong medication']
######CONCEPT EXTRACTED: ['the medical center

In [11]:
open("CPankow_Results.txt", "w+").write(str(hypernymsDict))

1968